In [1]:
!pip install seaborn
# !pip install tensorflow-io
!pip install keras --upgrade
%conda install -c hesi_m keras

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.

Note: you may need to restart the kernel to use updated packages.


WARNING conda.core.index:push_record(267): Skipping pypi/pypi::bleach-5.0.0-pypi_0 due to InvalidSpec: 1.1.0<1.2

InvalidVersionSpec: Invalid version '1.1.0<1.2': invalid character(s)



# setup

In [2]:
import os
import pathlib
import pickle

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow as tf
import librosa
import librosa.display
import pandas as pd

import tensorflow.python.keras as keras
#import tensorflow_io as tfio

from keras import layers
from keras import models
from IPython import display
from pathlib import Path
from IPython.display import Audio
from keras import utils

from sklearn.model_selection import train_test_split


# Set the seed value for experiment reproducibility.
seed = 42
SR=16000 # resampling as the ram can't handle this much calculation
BATCH_SIZE = 10
AUD_LENGTH = 10#sec
tf.random.set_seed(seed)
np.random.seed(seed)

# generating a dataset

In [3]:
# getting audio dataset path to divide into 3 datasets and also for making tf datasets later

DATASET_AUDIO_PATH = 'classwise_final_2k_imtiaz/classwise_final_2k'
class_names = os.listdir(DATASET_AUDIO_PATH)
print("Our class names: {}".format(class_names,))

audio_paths = []
labels = []
for label, name in enumerate(class_names):
    label = int(name)
    print("Processing speaker {}".format(name,))
    print("Actual Label ",label)
    dir_path = Path(DATASET_AUDIO_PATH) / name
    speaker_sample_paths = [
        os.path.join(dir_path, filepath)
        for filepath in os.listdir(dir_path)
        if filepath.endswith(".wav")
    ]
    audio_paths += speaker_sample_paths
    labels += [label] * len(speaker_sample_paths)

print(
    "Found {} files belonging to {} classes.".format(len(audio_paths), len(class_names))
)

Our class names: ['0', '1', '2', '3', '4', '5']
Processing speaker 0
Actual Label  0
Processing speaker 1
Actual Label  1
Processing speaker 2
Actual Label  2
Processing speaker 3
Actual Label  3
Processing speaker 4
Actual Label  4
Processing speaker 5
Actual Label  5
Found 10500 files belonging to 6 classes.


In [4]:
X_train, X, y_train, y = train_test_split(audio_paths, labels, test_size=0.3, random_state=seed)
X_val,X_test,y_val,y_test = train_test_split(X,y, test_size=0.333333, random_state=seed)

In [5]:
print(len(X_train),len(y_train))
print(len(X_val),len(y_val))
print(len(X_test),len(y_test))

7350 7350
2100 2100
1050 1050


In [6]:
# utility functions for repeating audio files
def repeated_data(file_path):
    """ This function will take a file path and give out truncated and padded to 10s version waveform"""
    y, sr = librosa.load(file_path,sr=SR)
    aud_length = AUD_LENGTH*sr # making all audio length 10 s and truncating the rest
    duration = librosa.get_duration(y=y, sr=sr)
    if duration < AUD_LENGTH:
        y = np.tile(y, int((aud_length/sr) // duration)+1)
    y = librosa.resample(y[:aud_length], orig_sr=sr, target_sr=SR)
    return y

def repeated_dataset(dataset):
    """ This function generated waveshape dataset"""
    new_ds = []
    for f in dataset:
        new_ds.append(repeated_data(f))
    return new_ds

## datagenerator

In [7]:
class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size= BATCH_SIZE, 
                 n_classes=6, shuffle=True):
        'Initialization'
        self.dim = AUD_LENGTH * SR
        self.batch_size = batch_size
        self.labels = labels
        self.shuffle = shuffle
        self.list_IDs = list_IDs
        self.on_epoch_end()

    def path_to_audio(self,path):
        """Reads and decodes an audio file."""
        return repeated_data(path)

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Generate data
        X, y = self.__data_generation(indexes)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples'
        
        X = []
        y = []
        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            _tempx = self.path_to_audio(self.list_IDs[ID])
            #_tempx = self.spect_audio(_tempx)
            X.append(_tempx)

            # Store class
            y.append(self.labels[ID])

        return np.reshape(np.array(X), (self.batch_size,SR*AUD_LENGTH,1)).astype(np.float32),np.array(y).astype(np.float32)

In [8]:
# generating tf datasets
train_ds = DataGenerator(X_train,y_train)

valid_ds = DataGenerator(X_val,y_val)

test_ds = DataGenerator(X_test,y_test)

In [9]:
def DilatedConvModule(xx, filters):
    
    xx1 = tf.keras.layers.Conv1D(filters, kernel_size = 9,dilation_rate=1, padding= "same")(xx)
    xx1 = tf.keras.layers.BatchNormalization()(xx1)
    xx1 = tf.keras.layers.ReLU()(xx1)
    
    xx2 = tf.keras.layers.Conv1D(filters, kernel_size = 7,dilation_rate=2, padding= "same")(xx1)
    xx2 = tf.keras.layers.BatchNormalization()(xx2)
    xx2 = tf.keras.layers.ReLU()(xx2)
    
    xx4 = tf.keras.layers.Conv1D(filters, kernel_size = 3,dilation_rate=4, padding= "same")(xx2)
    xx4 = tf.keras.layers.BatchNormalization()(xx4)
    xx4 = tf.keras.layers.ReLU()(xx4)
    
    xx8 = tf.keras.layers.Conv1D(filters, kernel_size = 1,dilation_rate=8, padding= "same")(xx4)
    xx8 = tf.keras.layers.BatchNormalization()(xx8)
    xx8 = tf.keras.layers.ReLU()(xx8)
    #concat
    yy = tf.keras.layers.Concatenate(axis=1)([xx1,xx2,xx4,xx8])
    yy = tf.keras.layers.BatchNormalization()(yy)
    yy = tf.keras.layers.ReLU()(yy)
    
    return yy

def build_model(input_shape, num_classes):
    inputs = tf.keras.layers.Input(shape=input_shape, name="input")
    x      = tf.keras.layers.Conv1D(16, kernel_size = 9,dilation_rate=1, padding= "same")(inputs)
    x      = tf.keras.layers.BatchNormalization()(x)
    x      = tf.keras.layers.ReLU()(x)
    x      = tf.keras.layers.MaxPool1D(pool_size = 4)(x)
    
    # stacked resnet modules
    # inc1
    x      = DilatedConvModule(x,32)
    x      = tf.keras.layers.MaxPool1D(pool_size = 4)(x)
    #x      = tf.keras.layers.Dropout(0.5)(x)
    # inc2
    x      = DilatedConvModule(x,64)
    x      = tf.keras.layers.MaxPool1D(pool_size = 4)(x)
    #x      = tf.keras.layers.Dropout(0.25)(x)
    # inc4
    x      = DilatedConvModule(x,128)
    x      = tf.keras.layers.MaxPool1D(pool_size = 4)(x)
   #x      = tf.keras.layers.Dropout(0.25)(x)
    # inc8
    x      = DilatedConvModule(x,256)
    x      = tf.keras.layers.MaxPool1D(pool_size =  x.shape[-1])(x)
    #x      = tf.keras.layers.Dropout(0.5)(x)
    
    x      = tf.keras.layers.GlobalAveragePooling1D()(x)
    
    x      = tf.keras.layers.Flatten()(x)
    x      = tf.keras.layers.Dense(64, activation="relu")(x)
    x      = tf.keras.layers.Dense(32, activation="relu")(x)
    x      = tf.keras.layers.Dropout(0.25)(x)
    
    outputs = tf.keras.layers.Dense(num_classes, activation="softmax", name="output")(x)
    
    return tf.keras.models.Model(inputs=inputs, outputs=outputs)


aud_length = 16000*AUD_LENGTH

model = build_model((aud_length, 1), len(class_names))

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 160000, 1)]  0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 160000, 16)   160         ['input[0][0]']                  
                                                                                                  
 batch_normalization (BatchNorm  (None, 160000, 16)  64          ['conv1d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 re_lu (ReLU)                   (None, 160000, 16)   0           ['batch_normalization[0][0]']

In [10]:
# configuring the run
# Compile the model using Adam's default learning rate
model.compile(
    optimizer="Adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

# Add callbacks:
# 'EarlyStopping' to stop training when the model is not enhancing anymore
# 'ModelCheckPoint' to always keep the model that has the best val_accuracy
weight_save_filename = "weight_inception_classwise2k_cv.h5"

lr_reduce = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, verbose=1, mode='min', min_lr=1e-9)
earlystopping_cb = tf.keras.callbacks.EarlyStopping(monitor='loss', min_delta=0.001, patience=10, mode='min', restore_best_weights=True)
mdlcheckpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    weight_save_filename, monitor="val_accuracy", save_best_only=True,save_weights_only=True)

# training

In [11]:
from sklearn.model_selection import KFold
EPOCHS=50
NFOLDS=5
folds = KFold(n_splits=NFOLDS)
splits = folds.split(audio_paths, labels)

def evaluate_model(X_train, X_val, y_train, y_val,j):
    
    train_ds = DataGenerator(X_train,y_train)
    valid_ds = DataGenerator(X_val,y_val)
    aud_length = AUD_LENGTH * SR
    model = build_model((aud_length, 1), len(class_names))
    #epochs = 1
    #batch_size = 50
    model.compile(
    optimizer="Adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    weight_save_filename = "weight_inception_classwise2k_cv"+str(j+3)+"fold_.h5"
    lr_reduce = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, verbose=1, mode='min', min_lr=1e-9)
    earlystopping_cb = tf.keras.callbacks.EarlyStopping(monitor='loss', min_delta=0.001, patience=10, mode='min', restore_best_weights=True)
    mdlcheckpoint_cb = tf.keras.callbacks.ModelCheckpoint(weight_save_filename, monitor="val_accuracy", save_best_only=True,save_weights_only=True, save_freq='epoch')
    
    model.load_weights('weight_inception_classwise2k_cv2fold_.h5')

    history = model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=valid_ds,
    callbacks=[lr_reduce,earlystopping_cb, mdlcheckpoint_cb],)

    

 
    _, val_acc = model.evaluate(valid_ds, verbose = 1)
    model.load_weights("./"+weight_save_filename) #
    model.save("model_inception_classwise2k_cv"+str(j+4)+"fold_.h5")
    return model, val_acc

fin_model = 1
cv_scores, model_history = list(), list()
train = audio_paths
targets = labels
for fold, (train_idx, val_idx) in enumerate(splits):
    X_train = []
    X_valid = []
    y_train = []
    y_valid = []
    for i in train_idx:
        X_train.append(train[i])
        y_train.append(targets[i])
    for j in val_idx:
        X_valid.append(train[j])
        y_valid.append(targets[j])
    #X_train, X_valid = train[], train[test_idx]
    #y_train, y_valid = targets[train_idx], targets[test_idx]
    print('-'*15, '>', f'Fold {fold+4}', '<', '-'*15)
    model, val_acc = evaluate_model(X_train, X_val, y_train, y_val,fold)
    print('>%.3f' % val_acc)
    cv_scores.append(val_acc)
    if val_acc == max(cv_scores):
        fin_model = model
    model_history.append(model)

--------------- > Fold 4 < ---------------
Epoch 1/50
840/840 [==============================] - 711s 835ms/step - loss: 0.0248 - accuracy: 0.9942 - val_loss: 37.9554 - val_accuracy: 0.8143 - lr: 0.0010
Epoch 2/50
840/840 [==============================] - 700s 834ms/step - loss: 0.0192 - accuracy: 0.9943 - val_loss: 44.2284 - val_accuracy: 0.7800 - lr: 0.0010
Epoch 3/50
840/840 [==============================] - 700s 834ms/step - loss: 0.0402 - accuracy: 0.9895 - val_loss: 24.5840 - val_accuracy: 0.8133 - lr: 0.0010
Epoch 4/50
840/840 [==============================] - 701s 835ms/step - loss: 0.0174 - accuracy: 0.9942 - val_loss: 24.4018 - val_accuracy: 0.8181 - lr: 0.0010
Epoch 5/50
840/840 [==============================] - 700s 833ms/step - loss: 0.0353 - accuracy: 0.9915 - val_loss: 41.8755 - val_accuracy: 0.8000 - lr: 0.0010
Epoch 6/50
840/840 [==============================] - 699s 832ms/step - loss: 0.0167 - accuracy: 0.9945 - val_loss: 38.2949 - val_accuracy: 0.8195 - lr: 0.00

In [ ]:
def ensemble_predictions(members, testX,testy=1):
    yhats = [model.predict(testX) for model in members]
    yhats = np.array(yhats)
    # sum across ensemble members
    summed = np.sum(yhats, axis=0)
    # argmax across classes
    result = np.argmax(summed, axis=1)
    return result

: 

In [ ]:
preds = ensemble_predictions(model_history, valid_ds)
preds

: 

In [ ]:
np.unique(preds) 

: 

In [ ]:
preds = ensemble_predictions(model_history, test_ds)
preds

: 

In [ ]:
np.unique(preds) 

: 

# evaluation

## librispeech

In [ ]:
eval_path ='../input/spcup2022-final-dataset/asv_libri_ds/asv_libri_ds/librispeech'
speaker_sample_paths = [
        os.path.join(eval_path, filepath)
        for filepath in os.listdir(eval_path)
        if filepath.endswith(".wav")
    ]
X_eval = []
X_eval += speaker_sample_paths

print(
    "Found {} files".format(len(X_eval))
)

: 

In [ ]:
# making a column of names
import os
n = len(X_eval)
filenames = []
for i in range(n):
    f_name, f_ext = os.path.splitext(X_eval[i])
    filenames.append(os.path.basename(f_name)+f_ext)

: 

In [ ]:
y_eval = [0]*len(X_eval)
print(len(X_eval),len(y_eval))

: 

In [ ]:
eval_ds = DataGenerator(X_eval,y_eval,shuffle=False)

: 

In [ ]:
pred_class = ensemble_predictions(model_history, eval_ds)

: 

In [ ]:
score_df = pd.DataFrame({'file':filenames,'pred_class':pred_class})
score_df.head()

: 

In [ ]:
score_df.pred_class.value_counts()

: 

In [ ]:
# converting dataframe to csv
score_df.to_csv('inceptionnetcv_2k_librispeech.csv', header=False, index=False)

: 

## asvspoof

In [ ]:
eval_path ='../input/spcup2022-final-dataset/asv_libri_ds/asv_libri_ds/asvspoof/wav' 
speaker_sample_paths = [ os.path.join(eval_path, filepath) 
                        for filepath in os.listdir(eval_path) if filepath.endswith(".wav") ] 
X_asv = [] 
X_asv += speaker_sample_paths
X_asv = X_asv[:30000]
label_asv = [3]*len(X_asv)
print( "Found {} files".format(len(X_asv)))
#_, X_eval, _, y_eval = train_test_split(X_asv, label_asv, test_size=0.03333, random_state=seed)
X_eval = X_asv
y_eval = label_asv
print(len(X_eval))
print(len(y_eval))

: 

In [ ]:
# making a column of names
import os
n = len(X_eval)
filenames = []
for i in range(n):
    f_name, f_ext = os.path.splitext(X_eval[i])
    filenames.append(os.path.basename(f_name)+f_ext)

: 

In [ ]:
asv_ds = DataGenerator(X_eval,[5]*len(X_eval),shuffle=False)

: 

In [ ]:
spred_class = ensemble_predictions(model_history, asv_ds)

: 

In [ ]:
asv_df = pd.DataFrame({'audio':X_eval[:len(spred_class)],'labels':spred_class})

: 

In [ ]:
asv_df.labels.value_counts()

: 

In [ ]:
#converting dataframe to csv
asv_df.to_csv('inceptionnetcv_2k_asvspoof.csv', header=False, index=False)

: 

# final eval

In [ ]:
eval_path ='../input/spcup2022-final-dataset/final_competition_eval/final_competition_eval'
speaker_sample_paths = [
        os.path.join(eval_path, filepath)
        for filepath in os.listdir(eval_path)
        if filepath.endswith(".wav")
    ]
X_eval = []
X_eval += speaker_sample_paths

print(
    "Found {} files".format(len(X_eval))
)

: 

In [ ]:
# making a column of names
import os
n = len(X_eval)
filenames = []
for i in range(n):
    f_name, f_ext = os.path.splitext(X_eval[i])
    filenames.append(os.path.basename(f_name)+f_ext)

: 

In [ ]:
libri_ds = DataGenerator(X_eval,[0]*len(X_eval),shuffle=False)

: 

In [ ]:
spred_class = ensemble_predictions(model_history, libri_ds)

: 

In [ ]:
libri_df = pd.DataFrame({'audio':X_eval[:len(spred_class)],'labels':spred_class})

: 

In [ ]:
libri_df.labels.value_counts()

: 

In [ ]:
#converting dataframe to csv
libri_df.to_csv('inceptionnetcv_2k_final_eval.csv', header=False, index=False)

: 

: 